# Clustering Crypto

In [185]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [186]:
# Load the crypto_data.csv dataset.
filePath = "cryptoData.csv"
cryptoDF = pd.read_csv(filePath)
cryptoDF.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [187]:
# Keep all the cryptocurrencies that are being traded.
cryptoDF = cryptoDF[cryptoDF['IsTrading'] == True]
cryptoDF

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [188]:
# Keep all the cryptocurrencies that have a working algorithm.
cryptoDF = cryptoDF[cryptoDF['Algorithm'].notnull()]
cryptoDF.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [189]:
#check col. count nulls

for column in cryptoDF.columns:
    print(f"column{column} has {cryptoDF[column].isnull().sum()} null values")


columnUnnamed: 0 has 0 null values
columnCoinName has 0 null values
columnAlgorithm has 0 null values
columnIsTrading has 0 null values
columnProofType has 0 null values
columnTotalCoinsMined has 459 null values
columnTotalCoinSupply has 0 null values


In [190]:
# Remove the "IsTrading" column. 
cryptoDF.pop("IsTrading")
cryptoDF.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [191]:
#removing dup unnamed
cryptoDF.pop("Unnamed: 0")
cryptoDF.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [192]:
# Remove rows that have at least 1 null value.
cryptoDF = cryptoDF.dropna()
cryptoDF

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [193]:
# Keep the rows where coins are mined.
cryptoDF = cryptoDF[cryptoDF["TotalCoinsMined"] > 0]
cryptoDF

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [194]:
# Create a new DataFrame that holds only the cryptocurrencies names.
nameDF = cryptoDF[['CoinName']]
nameDF.head()



,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [195]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
cryptoDF.pop("CoinName")
cryptoDF.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [196]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(cryptoDF,  columns=["Algorithm", "ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [197]:
# Standardize the data with StandardScaler().
clusteredScaled = StandardScaler().fit_transform(clusteredDF[['TotalCoinSupply', 'TotalCoinsMined']])
clusteredScaled


array([[-0.1528703 , -0.11710817],
       [-0.145009  , -0.09396955],
       [ 4.48942416,  0.52494561],
       ...,
       [-0.13217937, -0.09561336],
       [-0.15255998, -0.11694817],
       [-0.15285552, -0.11710536]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [198]:
# Using PCA to reduce dimension to three principal components
pca = PCA(n_components=3)




In [199]:
X_pca = pca.fit_transform(Xscaled)
X_pca

array([[-0.33659863,  1.05432797, -0.52014644],
       [-0.31991979,  1.05458172, -0.52051099],
       [ 2.31419244,  1.71556036, -0.55965502],
       ...,
       [ 0.33376317, -2.33498478,  0.2943906 ],
       [-0.16534297, -1.94216618,  0.4056856 ],
       [-0.29143606,  0.86744554, -0.21733551]])

In [200]:
# Create a DataFrame with the three principal components
pcaDF = pd.DataFrame(data= X_pca, columns=['PC 1', 'PC 2', 'PC 3'], index=X.index)
pcaDF.head(10)

,PC 1,PC 2,PC 3
0,-0.336599,1.054328,-0.520146
2,-0.319920,1.054582,-0.520511
5,2.314192,1.715560,-0.559655
7,-0.147076,-1.380656,0.091175
8,-0.154755,-2.060643,0.330037
9,-0.169134,-1.115395,-0.051650
10,-0.389384,1.272529,-0.393822
11,-0.156602,-2.127789,0.400356
12,-0.153196,-2.060736,0.330021
13,-0.165342,-1.942166,0.405686


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [201]:

inertia = []
k = list(range(1, 11))


#calc inertia
for i in k:
    km = KMeans(n_clusters = i, random_state = 0)
    km.fit(pcaDF)
    inertia.append(km.inertia_)


C:\Users\qatester\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [202]:
# Create an elbow curve to find the best value for K
elbowData = {"k": k, "inertia": inertia}
elbowDF = pd.DataFrame(elbowData)
elbowDF.hvplot.line(x = "k", y = "inertia", xticks = k, title = "Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [203]:
# Initialize the K-Means model.
 
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(pcaDF)

# Predict clusters
predictions = model.predict(pcaDF)
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [204]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clusteredDF = pd.concat([cryptoDF,pcaDF], axis=1, join="inner")


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clusteredDF["CoinName"] = nameDF

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions
clusteredDF["Class"] = model.labels_

# Print the shape of the clustered_df
print(clusteredDF.shape)
clusteredDF.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.336599,1.054328,-0.520146,42 Coin,0
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.319920,1.054582,-0.520511,404Coin,0
5,X13,PoW/PoS,2.927942e+10,314159265359,2.314192,1.715560,-0.559655,EliteCoin,0
7,SHA-256,PoW,1.792718e+07,21000000,-0.147076,-1.380656,0.091175,Bitcoin,1
8,Ethash,PoW,1.076842e+08,0,-0.154755,-2.060643,0.330037,Ethereum,1
9,Scrypt,PoW,6.303924e+07,84000000,-0.169134,-1.115395,-0.051650,Litecoin,1
10,X11,PoW/PoS,9.031294e+06,22000000,-0.389384,1.272529,-0.393822,Dash,0
11,CryptoNight-V7,PoW,1.720114e+07,0,-0.156602,-2.127789,0.400356,Monero,1
12,Ethash,PoW,1.133597e+08,210000000,-0.153196,-2.060736,0.330021,Ethereum Classic,1
13,Equihash,PoW,7.383056e+06,21000000,-0.165342,-1.942166,0.405686,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [205]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clusteredDF, 
    x = "PC 1", 
    y = "PC 2", 
    z = "PC 3",
    color = "Class",
    symbol = "Class",
    hover_name= 'CoinName',
    hover_data= ['Algorithm'],
    width = 800
)

fig.update_layout(legend = dict(x=0, y=1))
fig.show()


In [206]:
# Create a table with tradable cryptocurrencies
clusteredDF.hvplot.table(
    columns=['CoinName', 'Algorithm', 'ProofType','TotalCoinSupply','TotalCoinsMined', 'Class'],
    sortable=True, 
    selectable=True
)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [207]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clusteredDF)}  tradable cryptocurrencies.')

There are 532  tradable cryptocurrencies.


In [208]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clusteredScaled = StandardScaler().fit_transform(clusteredDF[['TotalCoinSupply', 'TotalCoinsMined']])
clusteredScaled

array([[-0.1528703 , -0.11710817],
       [-0.145009  , -0.09396955],
       [ 4.48942416,  0.52494561],
       ...,
       [-0.13217937, -0.09561336],
       [-0.15255998, -0.11694817],
       [-0.15285552, -0.11710536]])

In [218]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plotDF = pd.DataFrame(
    data=clusteredScaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clusteredDF.index.copy()
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plotDF['CoinName'] = clusteredDF['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plotDF['Class'] = clusteredDF['Class']

plotDF.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,-0.152870,-0.117108,42 Coin,0
2,-0.145009,-0.093970,404Coin,0
5,4.489424,0.524946,EliteCoin,0
7,-0.152560,-0.116715,Bitcoin,1
8,-0.152870,-0.114747,Ethereum,1
9,-0.151629,-0.115726,Litecoin,1
10,-0.152545,-0.116910,Dash,0
11,-0.152870,-0.116731,Monero,1
12,-0.149767,-0.114622,Ethereum Classic,1
13,-0.152560,-0.116946,ZCash,1


In [220]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plotDF.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
    by="Class",
    hover_cols="CoinName",
    xlabel="Total Cryptocurrency Coins Mined",
    ylabel="Total Cryptocurrency Coin Supply",
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)